In [320]:
# -*- coding: utf-8 -*-
"""
Created on Tue Dec  4 15:11:48 2018

@author: Fang
"""

import pandas as pd
import numpy as np
from sqlalchemy import *
from sqlalchemy import create_engine
import os
import datetime as dt
import time

module_dir = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\Modules\\DataCollection'
os.chdir(module_dir)


def hist_prices(ticker, start_date, end_date = dt.datetime.today()):
    start_date_unix = int(start_date.timestamp())
    end_date = dt.datetime.today()
    end_date_unix = int(end_date.timestamp())

    hist_price_url = 'https://query1.finance.yahoo.com/v8/finance/chart/{0}?symbol={0}&period1={1}&period2={2}&interval=1d'.format(ticker,start_date_unix,end_date_unix)

    with urlreq.urlopen(hist_price_url) as url:
        data = json.loads(url.read().decode())
        hist_prices = pd.DataFrame({**data['chart']['result'][0]['indicators']['quote'][0], **data['chart']['result'][0]['indicators']['adjclose'][0]},
                                        index = [dt.datetime.utcfromtimestamp(int(x)).date() for x in 
                                                 data['chart']['result'][0]['timestamp']])
        hist_prices.index = pd.to_datetime(hist_prices.index)
    return hist_prices

start_date = dt.datetime(1999,1,1)

# Initializing Stock Universe
os.chdir('..\\')
os.chdir('..\\')
os.chdir('..\\Data\\Stock Universe')
#tsx = pd.read_csv('TSX.csv')['Symbol']#.tolist()
nyse = pd.read_csv('NYSE.csv')['Symbol']#.tolist()
nasdaq = pd.read_csv('NASDAQ.csv')['Symbol']#.tolist()
amex = pd.read_csv('AMEX.csv')['Symbol']#.tolist()
us_names = pd.concat([nyse,nasdaq,amex]).drop_duplicates().values.tolist()

del nyse, nasdaq, amex

dbs_dir = 'C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\DBs'

os.chdir(dbs_dir)

earn_engine = create_engine('sqlite:///earningsHistory.db', echo=False)
yahoo_engine = create_engine('sqlite:///yahoo.db', echo = False)

run_update = True

def return_profiles(ticker):
    start_time = time.time()
    prices_query = 'SELECT Date, adjclose FROM historicalPrices WHERE Underlying = "{}"'.format(ticker)
    curr_closes = pd.read_sql_query(prices_query, con=hisprices_engine, index_col = 'Date')
    
    if len(curr_closes) == 0:
        return (ticker,)*8
    curr_closes.index = pd.to_datetime(curr_closes.index)

    yahoo_db_query = 'SELECT Underlying, industry, sector FROM profiles WHERE Underlying = "{}"'.format(ticker)
    curr_yahoo_db = pd.read_sql_query(yahoo_db_query, con = yahoo_engine, index_col = 'Underlying').reset_index().drop_duplicates('Underlying').reset_index(drop = True)
    if len(curr_yahoo_db) == 0:
        curr_yahoo_db = pd.DataFrame({'Underlying': [ticker], 'industry': [np.nan], 'sector': [np.nan]})
        
    quarter_mapping = {1: 1, 2:1, 3:1, 
                       4:2, 5:2, 6:2,
                       7:3, 8:3, 9:3,
                       10:4, 11:4, 12:4}

    curr_closes['year'] = curr_closes.index.year
    curr_closes['month'] = curr_closes.index.month
    curr_closes['boy'] = curr_closes['year'] - curr_closes['year'].shift(1)
    curr_closes['bom'] = curr_closes['month'] - curr_closes['month'].shift(1)
    curr_closes['boq'] = curr_closes['month'] % 3
    curr_closes['quarter'] = curr_closes['month'].replace(quarter_mapping)
    curr_closes['Q'] = curr_closes['year'].apply(lambda x: str(x)) + ' Q' + curr_closes['quarter'].apply(lambda x: str(x))
    curr_closes['dailyReturns'] = curr_closes['adjclose'].pct_change()

    annual_returns = curr_closes[curr_closes['boy'] != 0][['adjclose','year']]
    annual_returns['adjclose'] = annual_returns['adjclose'].pct_change().shift(-1)
    annual_returns = annual_returns.set_index('year').dropna()
    annual_returns.columns = ['return']

    quarterly_returns = curr_closes[curr_closes['bom'] != 0][['adjclose','boq','Q']]
    quarterly_returns = quarterly_returns[quarterly_returns['boq'] == 1][['adjclose','Q']]
    quarterly_returns['adjclose'] = quarterly_returns['adjclose'].pct_change().shift(-1)
    quarterly_returns = quarterly_returns.set_index('Q').dropna()
    quarterly_returns.columns = ['return']

    def risk_profiles(returns_df, curr_closes, curr_yahoo_db):
        maxDrawDowns = pd.DataFrame(index = returns_df.index, columns = ['maxDrawDowns'])
        downsideVols = pd.DataFrame(index = returns_df.index, columns = ['downsideVols'])
        returnVols = pd.DataFrame(index = returns_df.index, columns = ['vol'])

        for period in returns_df.index:
            curr_year = curr_closes[curr_closes[returns_df.index.name] == period][['adjclose','dailyReturns']]
            curr_year['cumReturns'] = (curr_year['dailyReturns'] + 1).cumprod()
            curr_year['rollingMax'] = curr_year['cumReturns'].expanding(min_periods=1).max()
            curr_year['drawDown'] = curr_year['cumReturns']/curr_year['rollingMax'] - 1
            curr_downside = curr_year[curr_year['dailyReturns'] < 0]

            maxDrawDowns.loc[period,'maxDrawDowns'] = curr_year['drawDown'].min()
            downsideVols.loc[period,'downsideVols'] = curr_downside['dailyReturns'].std()*np.sqrt(len(curr_downside))
            returnVols.loc[period, 'vol'] = curr_year['dailyReturns'].std()*np.sqrt(252)

        maxDrawDowns, downsideVols, returnVols = maxDrawDowns.T, downsideVols.T, returnVols.T
        maxDrawDowns.index, downsideVols.index, returnVols.index = curr_yahoo_db.Underlying, curr_yahoo_db.Underlying, curr_yahoo_db.Underlying
        maxDrawDowns[['industry','sector']], downsideVols[['industry','sector']], returnVols[['industry','sector']] = curr_yahoo_db[['industry','sector']], curr_yahoo_db[['industry','sector']], curr_yahoo_db[['industry','sector']]
        maxDrawDowns[['industry','sector']], downsideVols[['industry','sector']], returnVols[['industry','sector']] = curr_yahoo_db.iloc[:,1:].values, curr_yahoo_db.iloc[:,1:].values, curr_yahoo_db.iloc[:,1:].values
        return  maxDrawDowns, downsideVols, returnVols

    annualDDs, annualDownsides, annualVols = risk_profiles(annual_returns, curr_closes, curr_yahoo_db)
    quarterlyDDs, quarterlyDownsides, quarterlyVols = risk_profiles(quarterly_returns, curr_closes, curr_yahoo_db)
    annual_returns, quarterly_returns = annual_returns.T, quarterly_returns.T
    annual_returns.index, quarterly_returns.index = curr_yahoo_db.Underlying, curr_yahoo_db.Underlying
    annual_returns[['industry','sector']], quarterly_returns[['industry','sector']] = curr_yahoo_db[['industry','sector']], curr_yahoo_db[['industry','sector']]
    annual_returns[['industry','sector']], quarterly_returns[['industry','sector']] = curr_yahoo_db.iloc[:,1:].values, curr_yahoo_db.iloc[:,1:].values

    print("Completed in %s seconds" % (time.time() - start_time))
    
    return annual_returns, annualDDs, annualDownsides, annualVols, quarterly_returns, quarterlyDDs, quarterlyDownsides, quarterlyVols



if run_update:
    
    start_time = time.time()
    i = 0
    total_length = len(us_names)
    for ticker in us_names:
        

        try:
            curr_hist_prices = hist_prices(ticker, start_date)
            curr_hist_prices = curr_hist_prices.sort_index(ascending = True)
            curr_hist_prices['Underlying'] = ticker
        except:
            print('Failed prices for {}'.format(ticker))
            print('{0:.2f}% Completed'.format(i/total_length*100))
            continue

        table_in_db = True

        try:
            query = 'SELECT Date, Underlying FROM historicalPrices WHERE Underlying = "{}"'.format(ticker)
            curr_hist_db = pd.read_sql_query(query, con=hisprices_engine, index_col = 'Date').tail(1)
            curr_hist_db.index = pd.to_datetime(curr_hist_db.index)
        except:
            table_in_db = False

        if table_in_db and len(curr_hist_db) != 0:
            latest_date_in_db = curr_hist_db.index.tolist()[0]
            curr_hist_prices = curr_hist_prices[curr_hist_prices.index > latest_date_in_db]

        curr_hist_prices.to_sql('historicalPrices', con=hisprices_engine, 
                                if_exists='append', index_label = 'Date')
        i += 1
        print('{0:.2f}% Completed'.format(i/total_length*100))

    print("Completed in %s seconds" % (time.time() - start_time))

0.01% Completed
0.02% Completed
0.03% Completed
0.05% Completed
0.06% Completed
0.07% Completed
Failed prices for AAV
0.07% Completed
0.08% Completed
0.09% Completed
0.10% Completed
0.12% Completed
0.13% Completed
0.14% Completed
0.15% Completed
0.16% Completed
Failed prices for ABR-A
0.16% Completed
Failed prices for ABR-B
0.16% Completed
Failed prices for ABR-C
0.16% Completed
0.17% Completed
0.18% Completed
0.20% Completed
0.21% Completed
0.22% Completed
0.23% Completed
0.24% Completed
0.25% Completed
0.27% Completed
0.28% Completed
0.29% Completed
0.30% Completed
0.31% Completed
0.32% Completed
0.33% Completed
0.35% Completed
0.36% Completed
0.37% Completed
0.38% Completed
0.39% Completed
0.40% Completed
0.42% Completed
0.43% Completed
0.44% Completed
0.45% Completed
0.46% Completed
0.47% Completed
0.48% Completed
0.50% Completed
0.51% Completed
0.52% Completed
0.53% Completed
0.54% Completed
0.55% Completed
0.57% Completed
0.58% Completed
0.59% Completed
Failed prices for AFS-A
0.

3.49% Completed
3.50% Completed
3.51% Completed
Failed prices for BK-C
3.51% Completed
3.52% Completed
3.53% Completed
3.54% Completed
3.56% Completed
3.57% Completed
3.58% Completed
3.59% Completed
3.60% Completed
3.61% Completed
3.62% Completed
3.64% Completed
3.65% Completed
3.66% Completed
3.67% Completed
3.68% Completed
3.69% Completed
3.71% Completed
3.72% Completed
3.73% Completed
3.74% Completed
Failed prices for BML-G
3.74% Completed
Failed prices for BML-H
3.74% Completed
Failed prices for BML-J
3.74% Completed
Failed prices for BML-L
3.74% Completed
3.75% Completed
3.76% Completed
3.77% Completed
3.79% Completed
3.80% Completed
3.81% Completed
3.82% Completed
3.83% Completed
3.84% Completed
3.86% Completed
3.87% Completed
3.88% Completed
3.89% Completed
3.90% Completed
3.91% Completed
3.92% Completed
3.94% Completed
3.95% Completed
3.96% Completed
3.97% Completed
3.98% Completed
Failed prices for BRK.A
3.98% Completed
Failed prices for BRK.B
3.98% Completed
3.99% Completed
4

Failed prices for DD-B
7.34% Completed
7.35% Completed
7.36% Completed
7.38% Completed
7.39% Completed
Failed prices for DDR-A
7.39% Completed
Failed prices for DDR-J
7.39% Completed
Failed prices for DDR-K
7.39% Completed
7.40% Completed
7.41% Completed
7.42% Completed
7.43% Completed
7.45% Completed
7.46% Completed
7.47% Completed
7.48% Completed
7.49% Completed
7.50% Completed
7.51% Completed
7.53% Completed
7.54% Completed
7.55% Completed
7.56% Completed
Failed prices for DHC.X
7.56% Completed
7.57% Completed
7.58% Completed
7.60% Completed
7.61% Completed
7.62% Completed
7.63% Completed
7.64% Completed
7.65% Completed
7.66% Completed
7.68% Completed
7.69% Completed
7.70% Completed
7.71% Completed
7.72% Completed
7.73% Completed
Failed prices for DLN-A
7.73% Completed
7.75% Completed
7.76% Completed
7.77% Completed
Failed prices for DLR-C
7.77% Completed
Failed prices for DLR-G
7.77% Completed
Failed prices for DLR-H
7.77% Completed
Failed prices for DLR-I
7.77% Completed
Failed pr

11.54% Completed
11.55% Completed
11.57% Completed
11.58% Completed
Failed prices for GGP
11.58% Completed
Failed prices for GGP-A
11.58% Completed
11.59% Completed
Failed prices for GGT-B
11.59% Completed
Failed prices for GGT-E
11.59% Completed
11.60% Completed
Failed prices for GGZ-A
11.60% Completed
11.61% Completed
11.62% Completed
11.64% Completed
11.65% Completed
11.66% Completed
11.67% Completed
11.68% Completed
Failed prices for GIG.P
11.68% Completed
Failed prices for GIG.U
11.68% Completed
Failed prices for GIG.W
11.68% Completed
11.69% Completed
11.70% Completed
11.72% Completed
11.73% Completed
11.74% Completed
11.75% Completed
11.76% Completed
11.77% Completed
11.79% Completed
11.80% Completed
11.81% Completed
Failed prices for GLO-A
11.81% Completed
Failed prices for GLO-B
11.81% Completed
Failed prices for GLO-G
11.81% Completed
11.82% Completed
11.83% Completed
11.84% Completed
11.86% Completed
Failed prices for GLP-A
11.86% Completed
11.87% Completed
11.88% Completed


Failed prices for JW.B
15.25% Completed
15.26% Completed
15.27% Completed
15.28% Completed
15.29% Completed
15.31% Completed
15.32% Completed
15.33% Completed
15.34% Completed
15.35% Completed
15.36% Completed
15.38% Completed
15.39% Completed
15.40% Completed
15.41% Completed
15.42% Completed
15.43% Completed
15.44% Completed
Failed prices for KEY-I
15.44% Completed
Failed prices for KEY-J
15.44% Completed
15.46% Completed
15.47% Completed
15.48% Completed
15.49% Completed
15.50% Completed
15.51% Completed
Failed prices for KIM-I
15.51% Completed
Failed prices for KIM-J
15.51% Completed
Failed prices for KIM-K
15.51% Completed
Failed prices for KIM-L
15.51% Completed
Failed prices for KIM-M
15.51% Completed
15.53% Completed
15.54% Completed
Failed prices for KKR-A
15.54% Completed
Failed prices for KKR-B
15.54% Completed
15.55% Completed
15.56% Completed
15.57% Completed
15.58% Completed
15.60% Completed
Failed prices for KMI-A
15.60% Completed
15.61% Completed
15.62% Completed
15.63%

...

67.32% Completed
67.33% Completed
67.34% Completed
67.36% Completed
67.37% Completed
67.38% Completed
67.39% Completed
67.40% Completed
67.41% Completed
67.42% Completed
67.44% Completed
67.45% Completed
67.46% Completed
67.47% Completed
67.48% Completed
67.49% Completed
67.51% Completed
67.52% Completed
67.53% Completed
67.54% Completed
Failed prices for XCRA
67.54% Completed
67.55% Completed
67.56% Completed
67.57% Completed
67.59% Completed
67.60% Completed
67.61% Completed
67.62% Completed
67.63% Completed
67.64% Completed
67.66% Completed
67.67% Completed
67.68% Completed
67.69% Completed
67.70% Completed
67.71% Completed
67.72% Completed
67.74% Completed
Failed prices for XSPL
67.74% Completed
67.75% Completed
67.76% Completed
67.77% Completed
67.78% Completed
67.79% Completed
67.81% Completed
67.82% Completed
67.83% Completed
67.84% Completed
67.85% Completed
67.86% Completed
67.87% Completed
67.89% Completed
67.90% Completed
67.91% Completed
67.92% Completed
67.93% Completed
67

72.54% Completed
72.55% Completed
72.56% Completed
72.57% Completed
72.58% Completed
72.60% Completed
72.61% Completed
72.62% Completed
72.63% Completed
72.64% Completed
72.65% Completed
72.67% Completed
72.68% Completed
72.69% Completed
72.70% Completed
72.71% Completed
72.72% Completed
Failed prices for ECF-A
72.72% Completed
72.73% Completed
72.75% Completed
72.76% Completed
72.77% Completed
72.78% Completed
72.79% Completed
72.80% Completed
72.82% Completed
72.83% Completed
72.84% Completed
72.85% Completed
72.86% Completed
72.87% Completed
72.88% Completed
72.90% Completed
72.91% Completed
72.92% Completed
72.93% Completed
72.94% Completed
72.95% Completed
72.97% Completed
72.98% Completed
72.99% Completed
73.00% Completed
73.01% Completed
73.02% Completed
73.03% Completed
73.05% Completed
73.06% Completed
73.07% Completed
73.08% Completed
73.09% Completed
73.10% Completed
73.12% Completed
73.13% Completed
73.14% Completed
73.15% Completed
73.16% Completed
73.17% Completed
73.18% 

77.80% Completed
77.81% Completed
77.83% Completed
77.84% Completed
77.85% Completed
77.86% Completed
77.87% Completed
77.88% Completed
77.89% Completed
77.91% Completed
77.92% Completed
77.93% Completed
77.94% Completed
77.95% Completed
77.96% Completed
Failed prices for HLM.P
77.96% Completed
77.98% Completed
77.99% Completed
78.00% Completed
78.01% Completed
78.02% Completed
78.03% Completed
78.04% Completed
78.06% Completed
78.07% Completed
78.08% Completed
78.09% Completed
78.10% Completed
78.11% Completed
78.13% Completed
78.14% Completed
78.15% Completed
78.16% Completed
78.17% Completed
78.18% Completed
78.19% Completed
78.21% Completed
78.22% Completed
78.23% Completed
78.24% Completed
78.25% Completed
78.26% Completed
78.28% Completed
78.29% Completed
78.30% Completed
78.31% Completed
78.32% Completed
78.33% Completed
78.34% Completed
78.36% Completed
78.37% Completed
78.38% Completed
78.39% Completed
78.40% Completed
78.41% Completed
78.43% Completed
78.44% Completed
78.45% 

83.20% Completed
83.22% Completed
Failed prices for NSP.W
83.22% Completed
Failed prices for NSP.X
83.22% Completed
83.23% Completed
83.24% Completed
83.25% Completed
83.26% Completed
83.27% Completed
83.29% Completed
83.30% Completed
83.31% Completed
83.32% Completed
83.33% Completed
83.34% Completed
83.35% Completed
83.37% Completed
83.38% Completed
83.39% Completed
83.40% Completed
83.41% Completed
83.42% Completed
83.44% Completed
83.45% Completed
83.46% Completed
83.47% Completed
83.48% Completed
83.49% Completed
83.50% Completed
83.52% Completed
83.53% Completed
83.54% Completed
83.55% Completed
83.56% Completed
83.57% Completed
83.59% Completed
83.60% Completed
83.61% Completed
83.62% Completed
83.63% Completed
83.64% Completed
83.65% Completed
83.67% Completed
83.68% Completed
83.69% Completed
83.70% Completed
83.71% Completed
83.72% Completed
83.74% Completed
83.75% Completed
83.76% Completed
83.77% Completed
83.78% Completed
83.79% Completed
83.80% Completed
83.82% Completed


88.27% Completed
88.28% Completed
88.30% Completed
88.31% Completed
88.32% Completed
88.33% Completed
88.34% Completed
88.35% Completed
88.36% Completed
88.38% Completed
88.39% Completed
88.40% Completed
88.41% Completed
88.42% Completed
88.43% Completed
88.45% Completed
88.46% Completed
88.47% Completed
88.48% Completed
88.49% Completed
88.50% Completed
88.51% Completed
88.53% Completed
88.54% Completed
88.55% Completed
88.56% Completed
88.57% Completed
88.58% Completed
Failed prices for TAL.W
88.58% Completed
88.60% Completed
88.61% Completed
88.62% Completed
88.63% Completed
88.64% Completed
88.65% Completed
88.66% Completed
88.68% Completed
88.69% Completed
88.70% Completed
88.71% Completed
88.72% Completed
88.73% Completed
88.75% Completed
88.76% Completed
88.77% Completed
88.78% Completed
88.79% Completed
88.80% Completed
88.81% Completed
88.83% Completed
88.84% Completed
88.85% Completed
88.86% Completed
88.87% Completed
88.88% Completed
88.90% Completed
88.91% Completed
88.92% 

In [305]:
spx = hist_prices('^GSPC', start_date)
spx = spx.sort_index(ascending = True)
spx['Underlying'] = 'SPX'
spx = spx[['Underlying','adjclose']]
spx['industry'] = 'SP500'
spx['sector'] = 'Index'

In [321]:
start_time = time.time()
i = 0
total_length = len(us_names)

for ticker in us_names:
    annual_returns, annualDDs, annualDownsides, annualVols, quarterly_returns, quarterlyDDs, quarterlyDownsides, quarterlyVols = return_profiles(ticker)
    if type(annual_returns) != str and ticker != 'AAPL':
        annual_returns.to_sql('annualReturns', con=return_engine, 
                              if_exists='append', index_label = 'Underlying')
        annualDDs.to_sql('annualDDs', con=return_engine, 
                         if_exists='append', index_label = 'Underlying')
        annualDownsides.to_sql('annualDownsides', con=return_engine, 
                               if_exists='append', index_label = 'Underlying')
        annualVols.to_sql('annualVols', con=return_engine, 
                          if_exists='append', index_label = 'Underlying')
        
        quarterly_returns.to_sql('quarterlyReturns', con=return_engine, 
                                 if_exists='append', index_label = 'Underlying')
        quarterlyDDs.to_sql('quarterlyDDs', con=return_engine, 
                            if_exists='append', index_label = 'Underlying')
        quarterlyDownsides.to_sql('quarterlyDownsides', con=return_engine, 
                                  if_exists='append', index_label = 'Underlying')
        quarterlyVols.to_sql('quarterlyVols', con=return_engine, 
                             if_exists='append', index_label = 'Underlying')    
    i += 1
    print('{0:.2f}% Completed'.format(i/total_length*100))

print("Completed in %s seconds" % (time.time() - start_time))

Completed in 6.629115343093872 seconds
0.01% Completed
Completed in 6.561033725738525 seconds
0.02% Completed
Completed in 3.6761672496795654 seconds
0.03% Completed
Completed in 6.582365036010742 seconds
0.05% Completed
Completed in 6.130602121353149 seconds
0.06% Completed
Completed in 4.714388132095337 seconds
0.07% Completed
0.08% Completed
Completed in 6.535518169403076 seconds
0.09% Completed
Completed in 8.224777936935425 seconds
0.10% Completed
Completed in 5.481420278549194 seconds
0.12% Completed
Completed in 10.908202886581421 seconds
0.13% Completed
Completed in 8.351152420043945 seconds
0.14% Completed
Completed in 7.753258943557739 seconds
0.15% Completed
Completed in 7.596626281738281 seconds
0.16% Completed
Completed in 6.105450868606567 seconds
0.17% Completed
0.18% Completed
0.20% Completed
0.21% Completed
Completed in 6.9304563999176025 seconds
0.22% Completed
Completed in 6.886578798294067 seconds
0.23% Completed
Completed in 5.280179262161255 seconds
0.24% Complete

2.11% Completed
Completed in 4.730183362960815 seconds
2.12% Completed
2.14% Completed
2.15% Completed
Completed in 3.7130424976348877 seconds
2.16% Completed
Completed in 4.688432931900024 seconds
2.17% Completed
Completed in 4.636598587036133 seconds
2.18% Completed
Completed in 6.539513826370239 seconds
2.19% Completed
Completed in 3.6098570823669434 seconds
2.20% Completed
Completed in 3.676166296005249 seconds
2.22% Completed
Completed in 4.914879083633423 seconds
2.23% Completed
Completed in 3.701073169708252 seconds
2.24% Completed
Completed in 4.292539358139038 seconds
2.25% Completed
Completed in 6.40882682800293 seconds
2.26% Completed
Completed in 5.606007814407349 seconds
2.27% Completed
Completed in 3.9704105854034424 seconds
2.29% Completed
Completed in 4.724359035491943 seconds
2.30% Completed
Completed in 3.768916130065918 seconds
2.31% Completed
Completed in 4.410201549530029 seconds
2.32% Completed
Completed in 6.584388017654419 seconds
2.33% Completed
2.34% Completed

4.11% Completed
Completed in 4.457075834274292 seconds
4.12% Completed
4.13% Completed
Completed in 6.952430486679077 seconds
4.14% Completed
Completed in 4.5189385414123535 seconds
4.16% Completed
Completed in 3.8796229362487793 seconds
4.17% Completed
Completed in 5.64490818977356 seconds
4.18% Completed
Completed in 5.341710805892944 seconds
4.19% Completed
Completed in 5.730671405792236 seconds
4.20% Completed
Completed in 3.544100761413574 seconds
4.21% Completed
Completed in 5.276905059814453 seconds
4.22% Completed
Completed in 6.699063301086426 seconds
4.24% Completed
4.25% Completed
Completed in 3.947445869445801 seconds
4.26% Completed
Completed in 6.815217733383179 seconds
4.27% Completed
Completed in 6.5773773193359375 seconds
4.28% Completed
Completed in 6.115670919418335 seconds
4.29% Completed
Completed in 6.332059621810913 seconds
4.31% Completed
Completed in 6.521554708480835 seconds
4.32% Completed
Completed in 5.41853141784668 seconds
4.33% Completed
4.34% Completed


6.03% Completed
Completed in 6.525350332260132 seconds
6.04% Completed
Completed in 3.7938718795776367 seconds
6.05% Completed
Completed in 4.197167873382568 seconds
6.06% Completed
Completed in 5.756598711013794 seconds
6.07% Completed
Completed in 5.512228965759277 seconds
6.08% Completed
6.09% Completed
Completed in 6.480696201324463 seconds
6.11% Completed
Completed in 6.557483434677124 seconds
6.12% Completed
Completed in 3.5335533618927 seconds
6.13% Completed
Completed in 6.613310098648071 seconds
6.14% Completed
Completed in 4.0271992683410645 seconds
6.15% Completed
6.16% Completed
Completed in 6.585383653640747 seconds
6.18% Completed
Completed in 6.576440334320068 seconds
6.19% Completed
6.20% Completed
Completed in 6.533496141433716 seconds
6.21% Completed
Completed in 6.5505051612854 seconds
6.22% Completed
Completed in 6.587376594543457 seconds
6.23% Completed
Completed in 4.974672079086304 seconds
6.24% Completed
Completed in 5.542081356048584 seconds
6.26% Completed
Com

Completed in 5.384595632553101 seconds
8.08% Completed
Completed in 6.563469409942627 seconds
8.09% Completed
Completed in 5.716706991195679 seconds
8.10% Completed
Completed in 3.6172966957092285 seconds
8.11% Completed
Completed in 4.822103500366211 seconds
8.13% Completed
Completed in 6.556460857391357 seconds
8.14% Completed
Completed in 6.544494867324829 seconds
8.15% Completed
Completed in 6.483688116073608 seconds
8.16% Completed
Completed in 4.270336627960205 seconds
8.17% Completed
8.18% Completed
8.20% Completed
Completed in 6.591341257095337 seconds
8.21% Completed
Completed in 5.86129355430603 seconds
8.22% Completed
Completed in 6.572418212890625 seconds
8.23% Completed
Completed in 6.525543212890625 seconds
8.24% Completed
Completed in 5.7476232051849365 seconds
8.25% Completed
Completed in 6.568427324295044 seconds
8.27% Completed
Completed in 6.48564887046814 seconds
8.28% Completed
Completed in 6.581420421600342 seconds
8.29% Completed
Completed in 6.551307678222656 se

...

Completed in 4.824105978012085 seconds
91.54% Completed
91.55% Completed
91.56% Completed
91.57% Completed
91.58% Completed
91.60% Completed
91.61% Completed
91.62% Completed
91.63% Completed
Completed in 5.3137853145599365 seconds
91.64% Completed
Completed in 5.306805849075317 seconds
91.65% Completed
Completed in 5.8922388553619385 seconds
91.67% Completed
Completed in 5.6638219356536865 seconds
91.68% Completed
Completed in 4.7268595695495605 seconds
91.69% Completed
Completed in 4.399229526519775 seconds
91.70% Completed
Completed in 3.5434935092926025 seconds
91.71% Completed
Completed in 3.551492929458618 seconds
91.72% Completed
Completed in 3.654224395751953 seconds
91.73% Completed
Completed in 3.8355839252471924 seconds
91.75% Completed
Completed in 4.653553485870361 seconds
91.76% Completed
Completed in 4.161297082901001 seconds
91.77% Completed
Completed in 4.518885135650635 seconds
91.78% Completed
Completed in 4.331413745880127 seconds
91.79% Completed
Completed in 5.363

93.29% Completed
Completed in 3.7340216636657715 seconds
93.30% Completed
Completed in 4.213728189468384 seconds
93.32% Completed
Completed in 3.7509336471557617 seconds
93.33% Completed
Completed in 4.0351855754852295 seconds
93.34% Completed
Completed in 5.382574796676636 seconds
93.35% Completed
Completed in 4.812129497528076 seconds
93.36% Completed
Completed in 5.200121641159058 seconds
93.37% Completed
Completed in 3.931445598602295 seconds
93.39% Completed
Completed in 4.786672353744507 seconds
93.40% Completed
Completed in 4.943802118301392 seconds
93.41% Completed
Completed in 5.34170937538147 seconds
93.42% Completed
Completed in 5.27286958694458 seconds
93.43% Completed
Completed in 3.8736486434936523 seconds
93.44% Completed
Completed in 3.828784942626953 seconds
93.45% Completed
Completed in 3.8476758003234863 seconds
93.47% Completed
Completed in 5.480312824249268 seconds
93.48% Completed
Completed in 3.6212897300720215 seconds
93.49% Completed
Completed in 3.893585443496

Completed in 5.395577669143677 seconds
95.04% Completed
Completed in 4.813100099563599 seconds
95.05% Completed
Completed in 4.429152488708496 seconds
95.06% Completed
Completed in 3.5804288387298584 seconds
95.07% Completed
Completed in 6.515841960906982 seconds
95.08% Completed
Completed in 4.980667591094971 seconds
95.09% Completed
Completed in 4.659557104110718 seconds
95.11% Completed
Completed in 4.351266145706177 seconds
95.12% Completed
Completed in 4.643549680709839 seconds
95.13% Completed
Completed in 4.536863088607788 seconds
95.14% Completed
Completed in 5.382572650909424 seconds
95.15% Completed
Completed in 4.076076507568359 seconds
95.16% Completed
Completed in 5.524194717407227 seconds
95.17% Completed
Completed in 4.5119287967681885 seconds
95.19% Completed
Completed in 5.372629165649414 seconds
95.20% Completed
Completed in 5.588034391403198 seconds
95.21% Completed
Completed in 6.288209438323975 seconds
95.22% Completed
Completed in 6.3250532150268555 seconds
95.23%

96.72% Completed
Completed in 5.168169736862183 seconds
96.73% Completed
Completed in 4.209329843521118 seconds
96.74% Completed
Completed in 4.0551464557647705 seconds
96.76% Completed
Completed in 4.544840335845947 seconds
96.77% Completed
Completed in 3.7210466861724854 seconds
96.78% Completed
Completed in 3.9881958961486816 seconds
96.79% Completed
Completed in 6.5604517459869385 seconds
96.80% Completed
Completed in 3.6611785888671875 seconds
96.81% Completed
Completed in 5.496297836303711 seconds
96.83% Completed
Completed in 6.712040662765503 seconds
96.84% Completed
Completed in 3.8866045475006104 seconds
96.85% Completed
Completed in 3.6920957565307617 seconds
96.86% Completed
Completed in 4.632609128952026 seconds
96.87% Completed
Completed in 4.57573390007019 seconds
96.88% Completed
Completed in 4.9876580238342285 seconds
96.89% Completed
Completed in 5.312786817550659 seconds
96.91% Completed
Completed in 4.626654148101807 seconds
96.92% Completed
Completed in 5.086419820

98.42% Completed
Completed in 6.692095756530762 seconds
98.43% Completed
Completed in 3.7170310020446777 seconds
98.44% Completed
Completed in 3.845327615737915 seconds
98.45% Completed
Completed in 4.906874656677246 seconds
98.46% Completed
Completed in 5.672797441482544 seconds
98.48% Completed
Completed in 4.509962320327759 seconds
98.49% Completed
Completed in 5.8044726848602295 seconds
98.50% Completed
Completed in 5.327776193618774 seconds
98.51% Completed
Completed in 4.746511459350586 seconds
98.52% Completed
Completed in 4.733338832855225 seconds
98.53% Completed
Completed in 4.915850400924683 seconds
98.55% Completed
Completed in 5.427482843399048 seconds
98.56% Completed
Completed in 5.649857997894287 seconds
98.57% Completed
Completed in 5.675834655761719 seconds
98.58% Completed
Completed in 5.760623216629028 seconds
98.59% Completed
Completed in 4.845499038696289 seconds
98.60% Completed
Completed in 4.153887748718262 seconds
98.61% Completed
Completed in 4.92580008506774